# An Introduction to Value Function Iteration (Part 1)

Quentin Batista, The University of Tokyo

## Overview

1. [The Bellman Operator and the Value Function Iteration (VFI) algorithm](#The-Bellman-Operator)
2. [The approximate Bellman Operator and the Fitted Value Function Iteration (FVFI) algorithm](#The-Approximate-Bellman-Operator)
3. [Solving the cake eating problem with FVFI in Python](#Python-Implementation)
4. [The Curse of Dimensionality](#The-Curse-of-Dimensionality)
5. [Brief, high-level discussion of variants on the basic algorithm](#Variants-on-the-Basic-Algorithm)

## The Bellman Operator

We consider the following problem.

### Sequential Formulation

Let $X$ be a nonempty set.

$$\max_{\left\{ x_{t}\right\} _{t=1}^{\infty}}\sum_{t=0}^{\infty}\beta^{t}F\left(x_{t},x_{t+1}\right)$$

subject to: 

$$x_{t+1}\in\Gamma\left(x_{t}\right)\quad\text{and}\quad x_{0}:\mathrm{given}$$

for $t=0,1,\dots$.

- $x_{t}$: state vector
- $\Gamma:X\rightarrow X$: feasibility constraint (correspondence)
- $F:A\rightarrow\mathbb{R}$ where $A=\left\{ \left(x,y\right)\in X\times X:y\in\Gamma\left(x\right)\right\} $: one-period return function
- $\beta\in\left(0,1\right)$: discount factor


### Application to the Cake Eating Problem from the QuantEcon lecture

Consider the setup from the [Cake Eating Problem QuantEcon lecture](https://python-intro.quantecon.org/cake_eating_problem.html).


$$\max_{\left\{ c_{t}\right\} _{t=0}^{\infty}}\sum_{t=0}^{\infty}\beta^{t}u\left(c_{t}\right) \tag{1}$$

subject to:

$$
x_{t+1} = x_t - c_t
\quad \text{and} \quad
0\leq c_t\leq x_t \tag{2}
$$


for $t=0,1,\dots$.

- $x_{t}$: size of the cake at time $t$
- $c_{t}$: choice of consumption at time $t$
- Law of motion: $x_{t+1}=x_{t}-c_{t}$
- Time 0 endowment: $x_{0}=\bar{x}$
- $u\left(c\right)=\frac{c^{1-\gamma}}{1-\gamma}$: utility function
- $\beta$: discount factor
- $\Gamma\left(x\right)=\left\{ y\in\mathbb{R}_{+}:y=x-c\:\mathrm{for}\:\mathrm{some}\:c\in\left[0,x\right]\right\}  $
- $F\left(x,y\right)=u\left(x-y\right)$

### Some Helpful Assumptions

1. $X\subset\mathbb{R}^{N}$
2. $F$: continuous
3. $F$: bounded

*Note: many examples from Economics require some modifications to satisfy assumption 3.*

### Recursive Formulation

#### A crucial observation

Denote $\Pi\left(x_{0}\right)=\left\{ \left\{ x_{t}\right\} _{t=0}^{\infty}:x_{t+1}\in\Gamma\left(x_{t}\right),t=0,1,\dots\right\} $.

This is the set of feasible paths.

Let $U\left(\underline{x}\right)=\sum_{t=0}^{\infty}\beta^{t}F\left(x_{t},x_{t+1}\right)$ where $\underline{x}=\left(x_{0},x_{1},\dots\right)$ and $\underline{x}^{t}=\left(x_{t},x_{t+1},\dots\right)$.

Observe that:

$$\begin{eqnarray}
U\left(\underline{x}\right)&=&F\left(x_{0},x_{1}\right)+\sum_{t=1}^{\infty}\beta^{t}F\left(x_{t},x_{t+1}\right)\\&=&F\left(x_{0},x_{1}\right)+\beta U\left(\underline{x}^{1}\right)\\&=&F\left(x_{0},x_{1}\right)+\beta F\left(x_{0},x_{1}\right)+\beta^{2}U\left(\underline{x}^{2}\right)\\&\vdots&\\&=&\sum_{\tau=0}^{t-1}\beta^{\tau}F\left(x_{\tau},x_{\tau+1}\right)+\beta^{t}U\left(\underline{x}^{t}\right)
\end{eqnarray}$$

#### Optimal value function

Let: 

$$v^{*}\left(x\right)=\sup_{\underline{x}\in\Pi\left(x\right)}U\left(\underline{x}\right)$$

($v^{*}$ is well-defined and bounded by assumption 3.)

In the cake eating problem, $v^{*}$ is the maximum lifetime utility attainable from the current time when $x$ units of cake are left.

Using our "crucial observation":

$$\begin{eqnarray}
v^{*}\left(x\right)&=&\sup_{\underline{x}\in\Pi\left(x\right)}U\left(\underline{x}\right)\\&=&\sup_{\underline{x}\in\Pi\left(x\right)}F\left(x,y\right)+\beta U\left(\underline{x}^{1}\right)\\&=&\sup_{y\in\Gamma\left(x\right)}F\left(x,y\right)+\beta\sup_{\underline{x}\in\Pi\left(y\right)}U\left(\underline{x}\right)\\&=&\sup_{y\in\Gamma\left(x\right)}F\left(x,y\right)+\beta v^{*}\left(y\right)
\end{eqnarray}$$

This equation is called the Bellman equation.

### The Bellman Operator

Let $\mathcal{B}\left(X\right)$ be the set of bounded functions from $X$ to $\mathbb{R}$ and $\mathcal{C}_{b}\left(X\right)$ the set of bounded countinuous functions from $X$ to $\mathbb{R}$.

Given $v\in\mathcal{B}\left(X\right)$, let

$$w\left(x\right)\equiv\sup_{y\in\Gamma\left(x\right)}F\left(x,y\right)+\beta v\left(y\right)$$ 

One can show that $w\in\mathcal{B}\left(X\right)$.

We can regard the right-hand side of this equation as a function of $v$. Using this observation, we can define the operator $T:\mathcal{B}\left(X\right)\rightarrow\mathcal{B}\left(X\right)$ satisfying:

$$\left(Tv\right)\left(x\right)=\sup_{y\in\Gamma\left(x\right)}F\left(x,y\right)+\beta v\left(y\right)$$

This operator is called the Bellman operator.

(If $v\in\mathcal{C}_{b}\left(X\right)$, we can replace $\sup$ with $\max$ (by the Extreme Value Theorem) and $Tv\in\mathcal{C}_{b}\left(X\right)$ (by the Theorem of Maximum).)

### Value Function Iteration Algorithm

Observe that $Tv^{*}=v^{*}$, i.e. $v^{*}$ is a fixed point of $T$ 

**Idea:** Start with some function $v$ and apply $T$ repeatedly until convergence

<img src="vfi_algo.jpeg">

where $bS=\mathcal{B}\left(X\right)$ in our notation and $d_{\infty}\left(v,w\right)=\sup_{x\in X}\left|v\left(x\right)-w\left(x\right)\right|$. 

*Source: Chapter 5 of [1]*

A stopping criterion is used to avoid iterating for an infinite amount of time.

#### How does this work?

1. Show that $T$ is a **contraction mapping** in $\left(\mathcal{B}\left(X\right),d_{\infty}\right)$, i.e. $d_{\infty}\left(Tv,Tw\right)\leq\beta d_{\infty}\left(v,w\right)$
2. Show that $\left(\mathcal{B}\left(X\right),d_{\infty}\right)$ is a complete metric space (see [here](https://en.wikipedia.org/wiki/Complete_metric_space#Some_theorems))
3. Apply [Banach's fixed point theorem](https://en.wikipedia.org/wiki/Banach_fixed-point_theorem)

Result: $T^{n}v$ converges (uniformly) to $v^{*}$ as $n\rightarrow\infty$

#### Error bound and rate of convergence

Using the fact that $T$ is a contraction mapping:

$$\begin{eqnarray}
d_{\infty}\left(T^{2}v,Tv\right)&\leq&\beta d_{\infty}\left(Tv,v\right)\\d_{\infty}\left(T^{3}v,T^{2}v\right)&\leq&\beta d_{\infty}\left(T^{2}v,Tv\right)\\&\vdots&\\d_{\infty}\left(T^{n}v,T^{n-1}v\right)&\leq&\beta d_{\infty}\left(T^{n-1}v,T^{n-2}v\right)
\end{eqnarray}$$

This gives us some information about the speed of convergence of the algorithm and helps us form expectation about what a "normal" error pattern should look like. ($\beta$ is called a Lipschitz constant for $T$)

We can also show that:

$$d_{\infty}\left(v^{*},Tv\right)\leq\frac{\beta}{1-\beta}d_{\infty}\left(Tv,v\right)$$

This gives us a bound for the approximation error arising from using a stopping criterion.

### Policy function

Assume that the agent chooses $x_{t+1}$ according to $x_{t+1}=\sigma\left(x_{t}\right)$ for some  $\sigma:X \rightarrow X$.

$\sigma$ is called a **policy function**. 


Let $v_{\sigma}$ be the value associated with following policy $\sigma$.

A policy is feasible if it satisfies $\sigma\left(x\right)\in\Gamma\left(x\right)$ for all $x$. Let $\Sigma$ denote the set of all feasible policies.

Observe that:

$$v^{*}\left(x\right)=\sup\left\{ v_{\sigma}\left(x\right):\sigma\in\Sigma\right\} $$

$\sigma^{*}$ is called an optimal policy function if $v_{\sigma^{*}}=v^{*}$

As we did with the Bellman operator, we can define an operator $T_{\sigma}$ satisfying

$$\left(T_{\sigma}v\right)\left(x\right)=F\left(x,\sigma\left(x\right)\right)+\beta v\left(\sigma\left(x\right)\right)$$

Iteration on $T_{\sigma}$ converges to $v_{\sigma}$. 

A policy $\sigma$ is $v$-greedy if it satisfies

$$\sigma\left(x\right)\in\arg\max_{y\in\Gamma\left(x\right)}F\left(x,y\right)+\beta v\left(y\right)$$

## The Approximate Bellman Operator

Computers generally use floating point arithmetic (i.e. use a string of 1 and 0 of fixed length, say 32 or 64, to approximate real numbers) so some degree of approximation is always necessary.  

Computers have finite amount of memory so it is in general not possible to store $v\left(x\right)$ for all $x\in\mathbb{R}_{+}$.

#### Challenge: How can we represent $v$ on a computer?

- Parametric approximation to $v$, say with a polynomial function $w\left(y\right)=\sum_{i=0}^{n-1}a_{i}y^{i}$
- Consider only a finite set of points in $\mathbb{R_{+}}$

This generates additional **approximation** error

### Fitted Value Function Iteration

Let $\hat{T}$ be an approximate Bellman operator. 

We have the following decomposition: $\hat{T}=L\circ T$, i.e. $\hat{T}v=LTv$.

- $T$ is the Bellman operator.
- $L$ is an approximation operator

**Idea:** Iterate on the approximate Bellman operator until convergence.

<img src="fvfi_algo.jpeg">

*Source: Chapter 6 of [1]*

#### Fitted Value Function Iteration + Piecewise Constant Linear Interpolation + Grid

In this lecture, we'll mostly focus on the following scheme: 

- Equally spaced grid of points 
- Piecewise linear interpolation 
- Constant extrapolation

<img src="illustration_basic_scheme.jpeg" width='600'>

**FVFI specialized to the Cake Eating Problem:**

<img src="specialized_fvfi.png">


#### How do we know that value iteration will still work when there is approximation error?

If $L$ is **nonexpansive** on $\left(\mathcal{C}_{b}\left(X\right),d_{\infty}\right)$, $\hat{T}$ is a uniform contraction on $\left(\mathcal{C}_{b}\left(X\right),d_{\infty}\right)$ and the previous line of reasoning applies. (See [6])

$L$ is **nonexpansive** on $\left(S,d\right)$ if 

$$d\left(Lv,Lw\right)\leq d\left(v,w\right)$$ 

for all $v,w\in S$. 

The approximation scheme described above is nonexpansive.



## Some Comments on FVFI

#### Advantages

- Extremely versatile
- Theoretical properties
- Global solution

#### Disadvantages

- Slow (compared to say perturbation methods)
- Often suffers from the Curse of Dimensionality
- Interaction between different approximation at different levels can sometimes cause instability

## References

[1] John Stachurski, 2009. "Economic Dynamics: Theory and Computation," MIT Press Books  
[2] Johannes Brumm & Simon Scheidegger, 2017. "Using Adaptive Sparse Grids to Solve High‐Dimensional Dynamic Models," Econometrica  
[3] Artificial Intelligence as Structural Estimation: Economic Interpretations of Deep Blue, Bonanza, and AlphaGo, Mitsuru Igami  
[4] Richard S. Sutton and Andrew G. Barto. 2018. Reinforcement Learning: An Introduction. A Bradford Book, Cambridge, MA, USA.  
[5] [Daisuke Oyama's](http://www.oyama.e.u-tokyo.ac.jp/) excellent lecture notes on Dynamic Programming  
[6] John Stachurski, 2008. "Continuous State Dynamic Programming via Nonexpansive Approximation," Computational Economics  
[7] Yongyang Cai & Kenneth Judd, 2013. "Shape-preserving dynamic programming," Mathematical Methods of Operations Research  
[8] Cai, Yongyang & Judd, Kenneth. (2014). Advances in Numerical Dynamic Programming and New Applications. Handbook of Computational Economics.